## Implementation

In [1]:
import asyncio
from binance import AsyncClient, BinanceSocketManager
from binance.client import Client
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import datetime as dt # added
from collections import deque

In [2]:
import os
from dotenv import load_dotenv 
load_dotenv("../constants/.env")

True

In [3]:
# api_key = os.environ.get('BINANCE_TESNET_KEY')
# secret_key = os.environ.get('BINANCE_TESTNET_SECRET')

api_key = os.environ.get('BINANCE_KEY')
secret_key = os.environ.get('BINANCE_SECRET')


In [ ]:
is_test = True

In [5]:
client = Client(api_key = api_key, api_secret = secret_key, tld = "com", testnet = is_test)

In [6]:
# client.get_account()
client.get_account() # account details

{'makerCommission': 10,
 'takerCommission': 10,
 'buyerCommission': 0,
 'sellerCommission': 0,
 'commissionRates': {'maker': '0.00100000',
  'taker': '0.00100000',
  'buyer': '0.00000000',
  'seller': '0.00000000'},
 'canTrade': True,
 'canWithdraw': True,
 'canDeposit': True,
 'brokered': False,
 'requireSelfTradePrevention': False,
 'preventSor': False,
 'updateTime': 1729790306845,
 'accountType': 'SPOT',
 'balances': [{'asset': 'BTC', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'LTC', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'ETH', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'NEO', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'BNB', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'QTUM', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'EOS', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'SNT', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'BNT', 'free': '0.00000000', 'locke

In [13]:
# Inicializando as constantes para os sinais
SIGNAL_HIGH = 1
SIGNAL_87 = 2
SIGNAL_75 = 3
SIGNAL_62 = 4
SIGNAL_MID = 5
SIGNAL_37 = 6
SIGNAL_25 = 7
SIGNAL_12 = 8
SIGNAL_LOW = 9
NONE = 0




class Trade:
    def __init__(self, start_price, start_time, signal_up, signal_down):
        self.running = True
        self.count = 0
        self.trigger_type = NONE
        self.strategy = 0
        self.total_opened = 0
        
        self.start_price = start_price
        self.trigger_price = start_price

        self.SIGNAL_UP = signal_up
        self.SIGNAL_DOWN = signal_down
        self.result = 0.0
        self.end_time = start_time
        self.start_time = start_time
        
    def close_trade(self, time, result, trigger_price):
        self.running = False
        self.end_time = time
        self.trigger_price = trigger_price
        self.result = result

    def update(self, time, close, threshold_up):
        self.count += 1
        
        # Processamento de trades diretamente sem funções auxiliares
        def process_trade(signal_type, threshold_up):
            if signal_type == 'buy':
                
                if self.strategy >= threshold_up/100:
                    self.trigger_type = self.SIGNAL_UP
                    result = (close - self.start_price)
                    self.close_trade(time, result, close)

        # Verificação dos sinais de COMPRA
        if self.SIGNAL_UP == 1 or self.SIGNAL_DOWN == 1:
            process_trade('buy', threshold_up)





class LongShortTrader():
    
    def __init__(self, symbol, bar_length, ema_s, donchian_window, donchian_window_prev,
                 threshold_up, threshold_down, units, quote_units, position = 0):
        
        self.symbol = symbol
        self.bar_length = bar_length
        self.available_intervals = ["1m", "3m", "5m", "15m", "30m", "1h", "2h", "4h", "6h", "8h", "12h", "1d", "3d", "1w", "1M"]
        self.units = units
        self.quote_units = quote_units
        self.position = position
        self.trades = 0 
        self.trade_values = []

        self.opened_trades = deque()
        self.closed_trades = deque()

        self.SIGNAL_HIGH = 1
        self.SIGNAL_87 = 2
        self.SIGNAL_75 = 3
        self.SIGNAL_62 = 4
        self.SIGNAL_MID = 5
        self.SIGNAL_37 = 6
        self.SIGNAL_25 = 7
        self.SIGNAL_12 = 8
        self.SIGNAL_LOW = 9
        self.NONE = 0
        
        #*****************add strategy-specific attributes here******************
        # self.SMA_S = sma_s
        # self.SMA_M = sma_m
        # self.SMA_L = sma_l
        self.ema_s=ema_s
        self.donchian_window = donchian_window
        self.donchian_window_prev = donchian_window_prev
        self.threshold_up = threshold_up
        self.threshold_down = threshold_down
        #************************************************************************
    
    def start_trading(self, historical_days):
        
        self.twm = ThreadedWebsocketManager()
        self.twm.start()
        
        if self.bar_length in self.available_intervals:
            self.get_most_recent(symbol = self.symbol, interval = self.bar_length,
                                 days = historical_days)
            self.twm.start_kline_socket(callback = self.stream_candles,
                                        symbol = self.symbol, interval = self.bar_length)
        # "else" to be added later in the course 
    
    def get_most_recent(self, symbol, interval, days):
    
        now = datetime.now(dt.UTC) # new
        past = str(now - timedelta(days = days))

        bars = client.get_historical_klines(symbol = symbol, interval = interval,
                                            start_str = past, end_str = None, limit = 1000)
        df = pd.DataFrame(bars)
        df["Date"] = pd.to_datetime(df.iloc[:,0], unit = "ms")
        df.columns = ["Open Time", "Open", "High", "Low", "Close", "Volume",
                      "Clos Time", "Quote Asset Volume", "Number of Trades",
                      "Taker Buy Base Asset Volume", "Taker Buy Quote Asset Volume", "Ignore", "Date"]
        df = df[["Date", "Open", "High", "Low", "Close", "Volume"]].copy()
        df["Time"] = df["Date"]
        df.set_index("Date", inplace = True)
        for column in df.columns:
            df[column] = pd.to_numeric(df[column], errors = "coerce")
        df["Complete"] = [True for row in range(len(df)-1)] + [False]
        
        self.data = df
    
    def stream_candles(self, msg):
        
        # extract the required items from msg
        event_time = pd.to_datetime(msg["E"], unit = "ms")
        start_time = pd.to_datetime(msg["k"]["t"], unit = "ms")
        first   = float(msg["k"]["o"])
        high    = float(msg["k"]["h"])
        low     = float(msg["k"]["l"])
        close   = float(msg["k"]["c"])
        volume  = float(msg["k"]["v"])
        complete=       msg["k"]["x"]
    
     
        # print out
        print(".", end = "", flush = True) # just print something to get a feedback (everything OK) 

        # feed df (add new bar / update latest bar)
        self.data.loc[start_time] = [first, high, low, close, volume, start_time, complete]

        # prepare features and define strategy/trading positions whenever the latest bar is complete
        if complete == True:
            self.define_strategy()
            self.execute_trades()

            # print(self.prepared_data.columns,self.prepared_data.shape)
            
            last_time = self.prepared_data.Time.values[-1]
            last_close = self.prepared_data.Close.values[-1]
            last_ema = self.prepared_data.EMA_short.values[-1]
            last_high = self.prepared_data.donchian_high.values[-1]
            last_75 = self.prepared_data.donchian_75.values[-1]
            print(f"{last_time} - close = {last_close} | ema = {last_ema} | high = {last_high}  | 75 = {last_75}")

    
    # Função para processar os cruzamentos
    def process_cross_up(self, df, i, band, signal, last_band_cross_up, band_name):
        if last_band_cross_up != band_name:
            df.at[i, 'SIGNAL_UP'] = signal
            
            return band_name
        return last_band_cross_up
    
    def process_cross_down(self, df, i, band, signal, last_band_cross_down, band_name):
        if last_band_cross_down != band_name:
            df.at[i, 'SIGNAL_DOWN'] = signal
    
            return band_name
        return last_band_cross_down
    
    def define_strategy(self):
        
        donchian_low = df['donchian_low'].values
        close = df['Close'].values
        
        # Inicializa o ponto de referência com o primeiro valor de 'Close'
        last_price = df['Close'].iloc[0]
        
        # Garante que as colunas de sinal estão preenchidas com zeros
        df['SIGNAL_UP'] = 0
        df['SIGNAL_DOWN'] = 0
    
        for i in range(1, len(df)):
            # Calcula a variação percentual acumulada em relação ao último ponto de referência
            pct_change = (df['Close'].iloc[i] - last_price) / last_price * 100
            
            # Se a variação percentual ultrapassa o threshold de compra, sinaliza uma nova compra
            if pct_change >= self.threshold_up:
                df.at[i, 'SIGNAL_UP'] = 1  # Marca sinal de compra
                last_price = df['Close'].iloc[i]  # Atualiza o ponto de referência para o valor atual

            # Caso especial para donchian_low
            if close[i] < self.donchian_low[i-self.donchian_window_prev]:
                if abs(close[i] - last_price) >= last_price * (self.threshold_down/100):
                    df.at[i, 'SIGNAL_DOWN'] = 1
                    last_price = close[i]
    
        self.prepared_data = df.copy()
    
    def execute_trades(self): 

        for ind, ot in enumerate(self.opened_trades):
            ot.update(self.prepared_data.Time.values[-1], self.prepared_data.Close.values[-1], self.threshold_up)
            if ot.SIGNAL_UP != 0:
                print(20*"#", "Compra aqui", 20*"#")
                ot.strategy += 1 * self.prepared_data.returns.values[-1]
            
            if ot.running == False:
                print(20*"#", "Fecha Compra aqui", 20*"#")
                ot.strategy += ot.strategy * -0.00152
                self.closed_trades.append(ot)
            
            
            self.opened_trades = [x for x in self.opened_trades if x.running == True]

        if self.prepared_data.SIGNAL_UP.values[-1] == 1 or self.prepared_data.SIGNAL_DOWN.values[-1] == 1:
            self.opened_trades.append(Trade(self.prepared_data.Close.values[-1], 
                                            self.prepared_data.Time.values[-1], 
                                            self.prepared_data.SIGNAL_UP.values[-1],
                                            self.prepared_data.SIGNAL_DOWN.values[-1]))  
        
        # if self.prepared_data["position"].iloc[-1] == 1: # if position is long -> go/stay long
        #     if self.position == 0:
        #         order = client.create_order(symbol = self.symbol, side = "BUY", type = "MARKET", quoteOrderQty = self.quote_units)
        #         self.report_trade(order, "GOING LONG")  
        #     elif self.position == -1:
        #         order = client.create_order(symbol = self.symbol, side = "BUY", type = "MARKET", quoteOrderQty = self.quote_units)
        #         self.report_trade(order, "GOING NEUTRAL")
        #         time.sleep(1)
        #         order = client.create_order(symbol = self.symbol, side = "BUY", type = "MARKET", quoteOrderQty = self.quote_units)
        #         self.report_trade(order, "GOING LONG")
        #     self.position = 1
        # elif self.prepared_data["position"].iloc[-1] == 0: # if position is neutral -> go/stay neutral
        #     if self.position == 1:
        #         order = client.create_order(symbol = self.symbol, side = "SELL", type = "MARKET", quoteOrderQty = self.quote_units)
        #         self.report_trade(order, "GOING NEUTRAL") 
        #     elif self.position == -1:
        #         order = client.create_order(symbol = self.symbol, side = "BUY", type = "MARKET", quoteOrderQty = self.quote_units)
        #         self.report_trade(order, "GOING NEUTRAL") 
        #     self.position = 0
        # if self.prepared_data["position"].iloc[-1] == -1: # if position is short -> go/stay short
        #     if self.position == 0:
        #         order = client.create_order(symbol = self.symbol, side = "SELL", type = "MARKET", quoteOrderQty = self.quote_units)
        #         self.report_trade(order, "GOING SHORT") 
        #     elif self.position == 1:
        #         order = client.create_order(symbol = self.symbol, side = "SELL", type = "MARKET", quoteOrderQty = self.quote_units)
        #         self.report_trade(order, "GOING NEUTRAL")
        #         time.sleep(1)
        #         order = client.create_order(symbol = self.symbol, side = "SELL", type = "MARKET", quoteOrderQty = self.quote_units)
        #         self.report_trade(order, "GOING SHORT")
        #     self.position = -1
    
    def report_trade(self, order, going): 
        
        # extract data from order object
        side = order["side"]
        time = pd.to_datetime(order["transactTime"], unit = "ms")
        base_units = float(order["executedQty"])
        quote_units = float(order["cummulativeQuoteQty"])
        price = round(quote_units / base_units, 5)
        
        # calculate trading profits
        self.trades += 1
        if side == "BUY":
            self.trade_values.append(-quote_units)
        elif side == "SELL":
            self.trade_values.append(quote_units) 
        
        if self.trades % 2 == 0:
            real_profit = round(np.sum(self.trade_values[-2:]), 3) 
            self.cum_profits = round(np.sum(self.trade_values), 3)
        else: 
            real_profit = 0
            self.cum_profits = round(np.sum(self.trade_values[:-1]), 3)
        
        # print trade report
        print(2 * "\n" + 100* "-")
        print("{} | {}".format(time, going)) 
        print("{} | Base_Units = {} | Quote_Units = {} | Price = {} ".format(time, base_units, quote_units, price))
        print("{} | Profit = {} | CumProfits = {} ".format(time, real_profit, self.cum_profits))
        print(100 * "-" + "\n")

SyntaxError: invalid syntax (2569813114.py, line 224)

In [14]:
symbol = "BTCUSDT"
bar_length = "1m"

units = 0.001
quote_units = 10
position = 0
ema_s = 50
donchian_window = 60
donchian_window_prev = 5
threshold_up = 1
threshold_down = 1

In [15]:
client.get_symbol_info(symbol = "BTCUSDT") # information on symbol / pair

{'symbol': 'BTCUSDT',
 'status': 'TRADING',
 'baseAsset': 'BTC',
 'baseAssetPrecision': 8,
 'quoteAsset': 'USDT',
 'quotePrecision': 8,
 'quoteAssetPrecision': 8,
 'baseCommissionPrecision': 8,
 'quoteCommissionPrecision': 8,
 'orderTypes': ['LIMIT',
  'LIMIT_MAKER',
  'MARKET',
  'STOP_LOSS',
  'STOP_LOSS_LIMIT',
  'TAKE_PROFIT',
  'TAKE_PROFIT_LIMIT'],
 'icebergAllowed': True,
 'ocoAllowed': True,
 'otoAllowed': True,
 'quoteOrderQtyMarketAllowed': True,
 'allowTrailingStop': True,
 'cancelReplaceAllowed': True,
 'isSpotTradingAllowed': True,
 'isMarginTradingAllowed': True,
 'filters': [{'filterType': 'PRICE_FILTER',
   'minPrice': '0.01000000',
   'maxPrice': '1000000.00000000',
   'tickSize': '0.01000000'},
  {'filterType': 'LOT_SIZE',
   'minQty': '0.00001000',
   'maxQty': '9000.00000000',
   'stepSize': '0.00001000'},
  {'filterType': 'ICEBERG_PARTS', 'limit': 10},
  {'filterType': 'MARKET_LOT_SIZE',
   'minQty': '0.00000000',
   'maxQty': '110.42471794',
   'stepSize': '0.0000

In [16]:
trader = LongShortTrader(symbol = symbol, bar_length = bar_length, ema_s = 50, 
                         donchian_window = 100, donchian_window_prev = 5, 
                         threshold_up=1 , threshold_down=1, units = units, 
                         quote_units = quote_units, position = position)

In [17]:
stop_streaming = False # setting a stop_streaming variable (initially: False)

In [18]:
async def start_trading(historical_days):
    client = await AsyncClient.create()
    bm = BinanceSocketManager(client)
    
    if trader.bar_length in trader.available_intervals:
        trader.get_most_recent(symbol = trader.symbol, interval = trader.bar_length,
                             days = historical_days)
        ts = bm.kline_socket(symbol = trader.symbol, interval = trader.bar_length)        
    # "else" to be added later in the course
    
    async with ts as tscm:
        while True:
            res = await tscm.recv()
            trader.stream_candles(res)
            
            if stop_streaming:
                break

    await client.close_connection()
await start_trading(historical_days = 5/25)

.2024-10-25 02:35:00 - close = 67979.64 | ema = 67953.34516695273 | high = 68074.0  | 75 = 68014.5


/tmp/ipykernel_380903/1453580877.py:152: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2024-10-25 02:35:00' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  self.data.loc[start_time] = [first, high, low, close, volume, start_time, complete]


..........................2024-10-25 02:36:00 - close = 67990.0 | ema = 67954.78262507841 | high = 68074.0  | 75 = 68014.5
.........................2024-10-25 02:37:00 - close = 68009.99 | ema = 67956.94763998741 | high = 68074.0  | 75 = 68014.5
...........................2024-10-25 02:38:00 - close = 68000.0 | ema = 67958.63598267667 | high = 68074.0  | 75 = 68014.5
.........................2024-10-25 02:39:00 - close = 67980.45 | ema = 67959.49144156348 | high = 68074.0  | 75 = 68014.5
.........................2024-10-25 02:40:00 - close = 67979.9 | ema = 67960.29178368473 | high = 68074.0  | 75 = 68014.5
..........................2024-10-25 02:41:00 - close = 68000.0 | ema = 67961.8489807822 | high = 68074.0  | 75 = 68014.5
..........................2024-10-25 02:42:00 - close = 67983.86 | ema = 67962.71216447405 | high = 68074.0  | 75 = 68014.5
..........................2024-10-25 02:43:00 - close = 67964.26 | ema = 67962.77286434341 | high = 68074.0  | 75 = 68014.5
...............

CANCEL read_loop


CancelledError: 